![Workshop Banner](assets/S2_M2.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLDiego/SPE_GeoHackathon_2025/blob/dev/S2_M2_Advanced_agents.ipynb)

***

# Session 02 // Module 02: Advanced Agents for Geoscience (RAG + SQL Analyst)

In this module you’ll build an agentic system that can both:
- Retrieve domain knowledge from JSON + PDFs (RAG with citations)
- Analyze well production data from a CSV using SQL (DuckDB)

You’ll learn how LangChain Agents differ from fixed chains, how to create tools, and how to compose them into a single agent that chooses the right tool at runtime (ReAct).

## Learning Objectives
- Understand the think → act → observe loop for Agents
- Build a RAG tool that returns concise, cited answers
- Build a SQL analyst tool that computes stats over tabular data
- Combine tools into a master ReAct-style agent with routing
- Ship a simple Gradio chat UI

## What you’ll build
- A local vector DB (Chroma) indexing JSON + PDFs
- A SQL-powered CSV analyst (DuckDB) wrapped as a tool
- A ReAct agent that selects between tools (and can use both)
- An interactive UI for unified querying

## What we’ll use and why
- LangChain (agents, tools, LCEL)
  - Tools encapsulate capabilities; ReAct agents plan and call tools
- Hugging Face Transformers
  - Local LLM for generation (Phi-3 mini or similar)
- Chroma + sentence-transformers
  - Fast local embeddings + vector store for RAG
- DuckDB (in-memory SQL over DataFrames/CSV)
  - Fast analytics with familiar SQL, safe by constraining to SELECT-only
- Gradio
  - Lightweight, shareable UI

> Tip: Prefer running on Colab with a GPU. On Apple Silicon (MPS), 4-bit quantization (bitsandbytes) isn’t supported—use float16 and smaller models.

## 1. Introduction to Agents

Agents follow a loop:
- Think: reason about the question and plan
- Act: call a tool and pass it arguments
- Observe: read tool output, refine plan, repeat until done

How Agents differ from Chains:
- Chains are fixed graphs (e.g., retrieve → prompt → generate). Great for predictable, stable flows.
- Agents are dynamic planners. They choose tools, order, and parameters at runtime based on the query.

When to use which:
- Use a Chain when the path is known and stable (RAG).
- Use an Agent when users may ask diverse questions requiring different tools (RAG vs CSV) or multiple steps.

Safety notes:
- Tools can do arbitrary actions. Keep tool descriptions precise.
- If you allow code execution (Pandas agent), prefer sandboxed environments and read-only data for demos.

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> Snippet: Minimal Agent thought loop (conceptual)
```text
Question -> [Agent thinks]
Action: geoscience_retriever
Observation: "...cited context..."
[Agent thinks]
Action: csv_analyst
Observation: "...average water cut = 23.1%..."
Final Answer: "...concise synthesis + sources"
```


In [ ]:
# Download utils from GitHub
!wget -q --show-progress https://raw.githubusercontent.com/CLDiego/SPE_GeoHackathon_2025/refs/heads/dev/spe_utils.txt -O spe_utils.txt
!wget -q --show-progress -x -nH --cut-dirs=5 -i spe_utils.txt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Core LangChain + tooling
!pip -q install langchain langchain-core langchain-community langchain-huggingface langchain-text-splitters langchain-chroma
!pip -q install langchain-experimental gradio pandas duckdb

# HF + models + vecstore deps
!pip -q install transformers accelerate bitsandbytes huggingface_hub chromadb pypdf

# Web rendering for PDF scraping
!pip -q install playwright nest_asyncio
!python -m playwright install chromium

# Optional telemetry fixes in some envs
!pip -q install --upgrade opentelemetry-api opentelemetry-sdk

In [ ]:
# Hugging Face token (optional). On Colab, you can use google.colab.userdata; otherwise env var.
import os
HF_TOKEN = os.environ.get('HF_TOKEN', None)
try:
    from google.colab import userdata  # type: ignore
    HF_TOKEN = userdata.get('HF_TOKEN') or HF_TOKEN
except Exception:
    pass

from huggingface_hub import login
if HF_TOKEN:
    try:
        login(token=HF_TOKEN)
    except Exception:
        pass
print("HF token set:" , bool(HF_TOKEN))

## 2. Settings and Paths

We set paths for data and a persistent vector DB to speed up re-runs.

- WORKDIR/DATA_DIR/PDF_DIR: local folders for JSON/PDFs and outputs
- DB_DIR: persisted Chroma database
- MODEL_EMBED: fast sentence embedding model
- LLM_NAME: local LLM for generation (Phi-3 mini is a good balance)
- CHUNK_SIZE/CHUNK_OVERLAP: split documents for better retrieval
- TOP_K: number of chunks to retrieve per question

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> Hardware notes
- CUDA (Colab GPU): can use 4-bit quantization for 7–8B models.
- Apple MPS: use float16 (no 4-bit). Consider ≤2B models if memory is tight.
- CPU: use float32 and small models; expect slower generation.

> Key parameters
- CHUNK_SIZE/OVERLAP: start with 1000/200; adjust based on corpus density
- TOP_K: 4–8 is typical; reduce to limit noise, increase to avoid missing facts
- MODEL_EMBED: All-MiniLM-L6-v2 is fast and accurate for semantic search

> Token handling
- Set tokenizer.pad_token to eos_token if missing to avoid padding errors

In [ ]:
from pathlib import Path

WORKDIR = Path.cwd()
DATA_DIR = WORKDIR / "raw_data"
PDF_DIR = DATA_DIR / "pdfs"
DB_DIR = WORKDIR / "local_data" / "geo_vector_db_agents"
DB_DIR.parent.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
PDF_DIR.mkdir(parents=True, exist_ok=True)

# Models
MODEL_EMBED = "sentence-transformers/all-MiniLM-L6-v2"
LLM_NAME = "microsoft/Phi-3-mini-4k-instruct"

# Chunking + Retrieval
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
TOP_K = 6

# Dataset source (JSON) used in S2_M1
HF_DATASET = "GainEnergy/ogdataset"

## 3. Tool 1: Geoscience RAG Retriever

We refactor the RAG pipeline into functions and include both JSON and PDFs.

Pipeline steps:
1) Download/ensure JSON dataset locally (HF dataset snapshot)
2) Load JSON and PDFs into LangChain Documents with consistent metadata
3) Split into chunks (RecursiveCharacterTextSplitter)
4) Embed chunks and persist to Chroma
5) Build a retrieval chain with a concise, grounded prompt
6) Wrap it as a LangChain Tool returning an answer plus brief sources

> Key parameters (Chroma + Embeddings)
- model_name: sentence-transformers/all-MiniLM-L6-v2
- chunk_size/chunk_overlap: 1000/200
- persist_directory: speed up dev re-runs

> Prompt guidance (system)
- Be helpful and concise
- Use provided context; say “don’t know” if unclear
- Include brief source titles in the answer

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> Snippet: Consistent metadata for better citations
```python
# Ensure PDFs yield titles and file paths
for d in pages:
    d.metadata = {
        **(d.metadata or {}),
        "title": d.metadata.get("title") or p.stem,
        "file_path": str(p)
    }
```

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> Retrieval tips
- If answers are vague, preview retrieved chunks and reduce TOP_K
- If missing details, increase TOP_K or reduce chunk size for finer granularity
- Ensure titles/topics are present for readable citations

In [ ]:
from typing import List, Dict, Any
from pathlib import Path

from spe_utils import ensure_json_dataset, load_json_docs, load_pdf_docs

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


def build_vectorstore(docs: List[Document], persist_dir: Path, recreate: bool = True) -> Chroma:
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    chunks = splitter.split_documents(docs)
    print(f"Split {len(docs)} docs into {len(chunks)} chunks.")
    embeddings = HuggingFaceEmbeddings(model_name=MODEL_EMBED)
    if recreate and persist_dir.exists():
        import shutil
        shutil.rmtree(persist_dir)
    vs = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=str(persist_dir))
    print(f"Vector DB at {persist_dir}")
    return vs


def prepare_rag_corpus(data_dir: Path, pdf_dir: Path, db_dir: Path) -> Chroma:
    json_path = ensure_json_dataset(data_dir, HF_DATASET)
    print(f"Using JSON corpus: {json_path}")
    jdocs = load_json_docs(json_path)
    pdocs = load_pdf_docs(pdf_dir)
    print(f"Loaded {len(jdocs)} JSON docs; {len(pdocs)} PDF pages.")
    all_docs = jdocs + pdocs
    if not all_docs:
        raise RuntimeError("No documents found. Add PDFs to raw_data/pdfs or check JSON dataset.")
    return build_vectorstore(all_docs, db_dir, recreate=True)


vectorstore = prepare_rag_corpus(DATA_DIR, PDF_DIR, DB_DIR)

## 7. RAG Chain Assembly

We compose:
- retriever = vectorstore.as_retriever(k=TOP_K)
- prompt = ChatPromptTemplate(system + human with {context} and {input})
- document_chain = create_stuff_documents_chain(llm, prompt)
- rag_chain = create_retrieval_chain(retriever, document_chain)

Output shape:
- rag_chain returns a dict with keys: {"answer", "context"}
  - answer: model output grounded in context
  - context: retrieved Document list for citations

Tip:
- Adjust TOP_K and chunking if answers feel noisy or incomplete.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

def build_generation_pipeline(model_id: str):
    use_cuda = torch.cuda.is_available()
    use_mps = getattr(torch.backends, "mps", None) and torch.backends.mps.is_available()

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model_kwargs = {"device_map": "auto"}

    if use_cuda:
        try:
            from transformers import BitsAndBytesConfig
            quant = BitsAndBytesConfig(load_in_4bit=True)
            model_kwargs["quantization_config"] = quant
            print("CUDA: 4-bit quantization enabled.")
        except Exception:
            model_kwargs["torch_dtype"] = torch.float16
            print("CUDA: fallback to float16.")
    elif use_mps:
        model_kwargs["torch_dtype"] = torch.float16
        print("MPS: using float16 (bitsandbytes 4-bit not supported on MPS).")
    else:
        model_kwargs["torch_dtype"] = torch.float32
        print("CPU: using float32 (slow).")

    model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    gen_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        temperature=0.2,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        return_full_text=False,
    )
    llm = HuggingFacePipeline(pipeline=gen_pipe)
    chat_model = ChatHuggingFace(llm=llm)
    return llm, chat_model

llm, chat_model = build_generation_pipeline(LLM_NAME)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful geoscience assistant. Use the provided context to answer concisely. If unsure, say you don't know. Include brief source titles."),
    ("human", "Context:\n{context}\n\nQuestion: {input}")
])
document_chain = create_stuff_documents_chain(llm, rag_prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)
print("RAG chain ready.")

In [ ]:
from langchain.tools import tool
from langchain_core.documents import Document

@tool("geoscience_retriever")
def geoscience_retriever(query: str) -> str:
    """
    Use this tool for conceptual/qualitative geoscience questions. It queries an indexed corpus (JSON + PDFs)
    and returns a concise answer with brief citations. Input should be a standalone question.
    """
    try:
        result = rag_chain.invoke({"input": query})
    except Exception as e:
        return f"RAG error: {e}"
    answer = result.get("answer", "")
    ctx: List[Document] = result.get("context", [])
    cites = []
    seen = set()
    for d in ctx:
        md = d.metadata or {}
        title = md.get("title") or md.get("topic") or md.get("source") or md.get("file_path") or "Untitled"
        if title in seen:
            continue
        seen.add(title)
        cites.append(f"- {title}")
    if cites:
        answer = f"{answer}\n\nSources:\n" + "\n".join(cites)
    return answer or "No answer generated. Try rephrasing."

print("RAG tool 'geoscience_retriever' ready.")

## 8. Tool 2: SQL Data Analyst (DuckDB over CSV)

We analyze well_production.csv using SQL. The tool generates a single SELECT query (via the chat model), enforces safety constraints, executes it in DuckDB, and returns both the SQL and a compact result table.

Dataset schema (per row):
- date: YYYY-MM-DD (string)
- well_name: string
- oil_rate_bpd, water_rate_bpd, gas_rate_scfd: numeric
- water_cut_pct: numeric (pre-computed in the sample)

What the code does:
- ensure_sample_csv: creates a synthetic CSV (3 wells × 60 days) if missing
- duckdb.connect + con.register("well_prod", df): exposes the DataFrame as SQL table
- _schema_hint: produces a readable schema description for prompting
- SQL_GUIDE: system prompt with strict constraints (SELECT-only, table+columns only)
- _extract_sql: extracts and validates a single SELECT; blocks DDL/DML and multi-statements
- Fuzzy helpers: light auto-correction for column casing and near-miss well names
- csv_sql_analyst tool: formats messages, invokes the chat model to get SQL, executes in DuckDB, returns SQL + result

Safety constraints:
- Only SELECT queries are allowed (INSERT/UPDATE/DELETE/DDL blocked)
- Single statement only; early split on ";" and validation
- Limited output (head(20)) to keep chat concise

Prompting tips:
- Mention operations explicitly: “average,” “sum,” “by well,” “between dates”
- Dates are strings; cast if needed: CAST(date AS DATE)
- Column names are case sensitive; the tool attempts case correction but keep names exact

Examples:
- Average water cut for a well:
  SELECT AVG(water_cut_pct) FROM well_prod WHERE well_name = 'Well-15/9-F-1-C';
- Total oil across all wells:
  SELECT SUM(oil_rate_bpd) FROM well_prod;

In [ ]:
import pandas as pd
from pathlib import Path

from spe_utils import ensure_sample_csv

CSV_PATH = DATA_DIR / "well_production.csv"

ensure_sample_csv(CSV_PATH)
well_prod = pd.read_csv(CSV_PATH)
print(well_prod.head(3))

In [ ]:
well_prod[well_prod.well_name == 'Well-15/9-F-11-A'].water_cut_pct.mean()

In [ ]:
import duckdb
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool

from spe_utils.agents import _find_closest_match, _extract_sql

# Register the DataFrame as a DuckDB table
con = duckdb.connect(database=":memory:")
con.register("well_prod", well_prod)


def _schema_hint(df: pd.DataFrame, table_name: str) -> str:
    desc = {
        "date": "Date string in YYYY-MM-DD",
        "well_name": "Well name",
        "oil_rate_bpd": "Oil rate (barrels per day)",
        "water_rate_bpd": "Water rate (barrels per day)",
        "gas_rate_scfd": "Gas rate (scf per day)",
        "water_cut_pct": "Water cut (%)",
    }
    lines = []
    for c, t in df.dtypes.items():
        lines.append(f"- {c} ({str(t)}): {desc.get(c, '')}".rstrip(": "))
    return f"Table Name: {table_name}\nSchema:\n" + "\n".join(lines)


SQL_GUIDE = ChatPromptTemplate.from_messages([
    ("system",
     "You write a single DuckDB SQL SELECT query for the table provided.\n"
     "Constraints:\n"
     "- Use only the table and its columns as provided in the schema.\n"
     "- Use ONLY the provided table identifier.\n"
     "- Only return ONE SELECT statement. No CTEs with multiple statements, no DDL/DML.\n"
     "- Dates are strings 'YYYY-MM-DD'. If needed, use CAST(date AS DATE).\n"
     "- Prefer standard SQL: WHERE, GROUP BY, ORDER BY, LIMIT, AVG, SUM, MIN, MAX, COUNT.\n"
     "- Column names are case sensitive as given. If the user uses a column name that is not in the schema, find the closest matching column name from the schema and use that instead.\n"
     "- For string values in WHERE clauses (like well names), if the user's value is not an exact match to existing values, try to find the closest existing value in that column and use that. Provide a note if a correction was made.\n"
     "{schema}\n"
     "Example 1 (average water cut for a well): Given a table named 'well_prod' with a 'well_name' column, and the question 'average water cut for Well-15/9-F-1-C'\n"
     "SELECT AVG(water_cut_pct) FROM well_prod WHERE well_name = 'Well-15/9-F-1-C';\n"
     "Example 2 (total oil rate across all wells): Given a table named 'well_prod' with an 'oil_rate_bpd' column, and the question 'total oil rate'\n"
     "SELECT SUM(oil_rate_bpd) FROM well_prod;"
    ),
    ("human", "Question: {question}\nReturn only the SQL.")
])


@tool("csv_sql_analyst")
def csv_sql_analyst(question: str) -> str:
    """
    Use this tool for quantitative questions over well_production via SQL (DuckDB).
    It dynamically generates a single SELECT query and executes it safely.
    """
    try:
        msgs = SQL_GUIDE.format_messages(schema=_schema_hint(well_prod, "well_prod"), question=question)
        resp = chat_model.invoke(msgs)
        sql = _extract_sql(
            getattr(resp, "content", str(resp)),
            table_columns=list(well_prod.columns),
            string_value_domains={"well_name": well_prod["well_name"].unique().tolist()},
        )
    except Exception as e:
        return f"SQL generation error: {e}"

    try:
        df_res = con.execute(sql).df()
        if df_res.empty:
            tbl = "(no rows)"
        else:
            tbl = df_res.head(20).to_string(index=False)
        return f"SQL:\n{sql}\n\nResult:\n{tbl}"
    except Exception as e:
        return f"SQL execution error: {e}"


print("CSV tool 'csv_sql_analyst' ready (DuckDB).")

## 5. Master Agent (Tool Routing)

Now we assemble the master agent. This agent's primary job is not to answer questions directly, but to **route the user's query to the correct tool**. It has access to both the `geoscience_retriever` (for conceptual questions) and the `csv_analyst` (for data questions).

### 5.1. Agent Initialization

We use `initialize_agent` to create our master agent. The key is to provide it with the list of available tools and a carefully crafted system prompt.

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/write.svg" width="20"/> **The System Prompt is Critical**
> The `system_prompt` acts as the agent's instruction manual. It must clearly and concisely describe what each tool does. The agent will read this prompt and the docstrings of each tool to decide which one to use. A well-written prompt is the most important factor for successful tool routing.

### 5.2. Agent Type

We use `AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION`, which is a powerful agent type designed for chat models and capable of handling tools with multiple inputs. It follows the ReAct (Reason+Act) framework, which makes its decision-making process transparent. When you set `verbose=True`, you will see the agent's "chain of thought" as it reasons about which tool to use, executes it, and observes the outcome.

## 5. Master Agent (tool routing)

We combine both tools and let a ReAct agent pick the right one at runtime.

- Tools: geoscience_retriever (RAG) and csv_analyst (Pandas)
- Model: chat_model (chat wrapper over HF pipeline)
- Agent type: STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
  - Produces a structured thought + tool call plan
  - More robust formatting than plain Zero-Shot

System message guidance:
- Route conceptual/qualitative questions to geoscience_retriever
- Route numeric/CSV questions to csv_analyst
- If both apply, run them in sequence and synthesize the final answer
- Keep answers concise; include sources for RAG

> Tool selection heuristics
- If the question mentions “average/mean/sum/min/max”, “filter”, or “by well/date”, prefer csv_analyst
- If the question asks “explain/define/describe”, prefer geoscience_retriever
- For combined tasks (e.g., “Summarize geology and total oil”), call both

> Debugging
- Set verbose=True to see the ReAct trace (Thought/Action/Observation)
- If the agent hallucinates tool names, tighten tool descriptions and system prompt

In [ ]:
from langchain.agents import initialize_agent, AgentType

tools = [geoscience_retriever, csv_sql_analyst]

system_prompt = (
    "You are a geoscience assistant.\n"
    "- Use 'geoscience_retriever' for conceptual or qualitative questions about geology, geophysics, drilling, reservoir engineering, etc., grounded in the indexed corpus (JSON + PDFs).\n"
    "- Use 'csv_sql_analyst' for questions that require computing statistics or filtering rows from the well_production.csv dataset via SQL.\n"
    "If both are relevant, use them in sequence and combine results. Be concise and include brief sources for RAG answers."
)

agent = initialize_agent(
    tools=tools,
    llm=chat_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={"system_message": system_prompt},
)
print("Master agent ready.")

## 6. Testing the Agent

Try three scenarios:
- RAG: “Explain the concept of seismic resolution.”
- CSV: “What is the average water cut for Well-15/9-F-1 C?”
- Combined: “Summarize the geology of the Volve field and tell me the total oil production.”

Interpreting output:
- With verbose=True, you’ll see the agent’s reasoning steps and tool calls
- RAG answers should end with brief Sources (titles)
- CSV answers should include the numeric result and minimal explanation

> <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> Tips
- If the agent times out, reduce max_iterations for csv agent and shorten max_new_tokens for the LLM
- If tool routing seems off, make tool descriptions more explicit and add examples in the system message

In [ ]:
tests = [
    "how many rows in the well_production.csv dataset has?",
    "What is the average water cut pct for Well-15/9-F-11-A in the dataframe?",
    "Summarize the geology of the Volve field and tell me the total oil production.",
]

# Optional: scrape a web page to PDF to ensure at least one PDF exists
try:
    from spe_utils import webpage_to_pdf
    sample_url = "https://en.wikipedia.org/wiki/Seismic_resolution"
    sample_pdf = PDF_DIR / "seismic_resolution_sample.pdf"
    if not sample_pdf.exists():
        print("Scraping sample web page to PDF ...")
        webpage_to_pdf(sample_url, sample_pdf)
        print(f"Saved: {sample_pdf}")
        # Rebuild vectorstore to index the new PDF
        vectorstore = prepare_rag_corpus(DATA_DIR, PDF_DIR, DB_DIR)
        retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})
        from langchain.chains.combine_documents import create_stuff_documents_chain
        from langchain.chains import create_retrieval_chain
        from langchain_core.prompts import ChatPromptTemplate
        rag_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful geoscience assistant. Use the provided context to answer concisely. If unsure, say you don't know. Include brief source titles."),
            ("human", "Context:\n{context}\n\nQuestion: {input}")
        ])
        document_chain = create_stuff_documents_chain(llm, rag_prompt)
        rag_chain = create_retrieval_chain(retriever, document_chain)
except Exception as e:
    print("Web-to-PDF step skipped:", e)

for q in tests:
    print("\n=== Q:", q)
    try:
        ans = agent.invoke(q)
        print(ans)
    except Exception as e:
        print("Agent error:", e)

## 7. Gradio UI Integration

We wrap the master agent in a small chat UI.

UI pieces:
- respond(message, history): calls agent.run and appends reply
- Chatbot: keeps the conversation visible
- Textbox + Buttons: send and clear
- demo.launch(share=False): local serving

> Deployment notes
- In some environments you may need: demo.launch(server_name="0.0.0.0", server_port=7860)
- share=True gets a temporary public URL (avoid for sensitive data)

> UX tips
- Keep answers concise; show citations for RAG in the message or a side panel
- For longer analyses, stream partial results or show “running…” status

In [ ]:
import gradio as gr
from typing import List, Tuple
from pathlib import Path


def respond(message: str, history: List[Tuple[str, str]]):
    if not message.strip():
        return "", history
    try:
        reply = agent.run(message)
    except Exception as e:
        reply = f"Agent error: {e}"
    history = history + [(message, reply)]
    return "", history


def ingest_pdf(file_obj) -> str:
    try:
        import shutil
        from spe_utils import load_pdf_docs
        # Save uploaded PDF into PDF_DIR
        dest = PDF_DIR / Path(file_obj.name).name
        shutil.copy(file_obj.name, dest)
        # Load and add to vectorstore
        docs = load_pdf_docs(PDF_DIR)
        # Rebuild vectorstore minimally: append mode not shown in quick demo, rebuild instead
        global vectorstore, retriever, rag_chain
        vectorstore = prepare_rag_corpus(DATA_DIR, PDF_DIR, DB_DIR)
        retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})
        print("Vectorstore updated after PDF upload.")
        return f"Uploaded and indexed: {dest.name}"
    except Exception as e:
        return f"PDF ingest error: {e}"


with gr.Blocks(title="Geo Agent: RAG + CSV Analyst") as demo:
    gr.Markdown("## Geo Agent: RAG + CSV Analyst\nAsk conceptual questions or data questions about well_production.csv.")
    chatbot = gr.Chatbot(height=420)
    with gr.Row():
        msg = gr.Textbox(placeholder="Ask about seismic resolution or well production stats...", scale=4)
        send = gr.Button("Send", variant="primary")
        clear = gr.Button("Clear")
    with gr.Row():
        uploader = gr.File(label="Upload PDF to index", file_types=[".pdf"], interactive=True)
        upload_btn = gr.Button("Ingest PDF")

    send.click(respond, [msg, chatbot], [msg, chatbot])
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: [], outputs=chatbot)
    upload_btn.click(lambda f: ingest_pdf(f), inputs=[uploader], outputs=[])

try:
    demo.launch(share=False)
except Exception as e:
    print("Gradio failed to launch:", e)

## 12. Troubleshooting and Tips

RAG quality
- Empty/weak answers: preview retrieved chunks; tune TOP_K; reduce CHUNK_SIZE or increase OVERLAP
- Irrelevant context: lower TOP_K or adjust splitter to improve chunk boundaries
- Missing titles in citations: verify metadata fields (“title”, “topic”, “file_path”)

Generation performance
- CUDA OOM: lower max_new_tokens; use 4-bit on CUDA; switch to a smaller LLM
- MPS (macOS): no bitsandbytes 4-bit; use float16; consider ≤2B models
- CPU: float32 is slow—minimize tokens and test with tiny models

SQL tool
- Parsing errors: keep questions specific; include columns/filters explicitly
- Case sensitivity: column names are case sensitive; prefer exact names
- Dates: stored as strings; cast when needed (CAST(date AS DATE))
- No rows returned: check well_name spelling; the tool applies fuzzy correction but verify input
- Security: SELECT-only guard blocks multi-statements and DDL/DML

Agent routing
- Strengthen tool descriptions and system guidance
- Provide examples of when to use each tool
- If the agent mixes Final Answer with Action, remind it to separate steps (ReAct discipline)

HF token and access
- Colab: from google.colab import userdata; userdata.get("HF_TOKEN")
- macOS zsh: echo 'export HF_TOKEN=YOUR_TOKEN' >> ~/.zshrc && source ~/.zshrc

Quick knobs to tweak
```python
# Retrieval
TOP_K = 4  # try 4–8
CHUNK_SIZE = 800
CHUNK_OVERLAP = 160

# Generation
temperature = 0.2
max_new_tokens = 192

# SQL display
MAX_ROWS = 20  # head limit for chat output
```